# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

##  Открыть файл с данными и изучить общую информацию. 

In [1]:
import pandas as pd
import math

In [2]:
data = pd.read_csv('data.csv')

In [3]:
display(data.head(5))
print(data.info())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None


---
В данных есть пропущенные значения в столбцах **days_employed** и **total_income**. В дальнейшем необходимо произвести анализ пропущенных значений и варианты их восстановления, если это будет необходимо для решения задачи.


Рассмотрим более подробно данные из всех столбцов датафрейма.

In [4]:
print(data['children'].unique())
print(data[(data['children'] == -1)]['children'].count())
print(data[(data['children'] == 20)]['children'].count())

[ 1  0  3  2 -1  4 20  5]
47
76


Столбец **Children** помимо "обычных" значений (0,1,2 .. 5) принимает значения -1 и 20, 47 и 76 раз соответственно. 
Предположительно -1 - означает, что клиент не имеет детей, а 20 - то, что кол-во детей более 5.

In [5]:
print(data[(data['days_employed'] < 0)]['days_employed'].count())

15906


Столбец **days_employed** - представляет вещественные значения,содержит пропущенные данные, а также часть значений отрицательна
(т.к. отрицательных значений много, то ошибка не случайна, где-то допускалась систематическая ошибка при записи чисел

In [6]:
print(data['dob_years'].unique())
print(data[(data['dob_years'] == 0)]['dob_years'].count())

[42 36 33 32 53 27 43 50 35 41 40 65 54 56 26 48 24 21 57 67 28 63 62 47
 34 68 25 31 30 20 49 37 45 61 64 44 52 46 23 38 39 51  0 59 29 60 55 58
 71 22 73 66 69 19 72 70 74 75]
101


Столбец **dob_years** содежит 101 элемент равных 0, скорее всего по каким-то причинам информация о возрасте клиента была потеряна.

In [7]:
print(data['education'].unique())

['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень']


Столбец **education** содержит элементы разного регистра, далее необходимо избавиться от дубликатов

In [8]:
print(data['family_status'].unique())

['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем']


In [9]:
print(data['gender'].unique())

['F' 'M' 'XNA']


Столбец **gender** помимо значений 'F' и 'M' содержит один элемент 'XNA' - предположительно информация о гендерной принадлежности клиента была потеряна.

In [10]:
print(data['income_type'].unique())

['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете']


In [11]:
print(data['debt'].unique())

[0 1]


In [12]:
print(data['total_income'].min())
print(data['total_income'].max())

20667.26379327158
2265604.028722744


In [13]:
print(data['purpose'].unique())

['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образованием'
 'сделка с подержанным автомобилем' 'получение образования' 'автомобиль'
 'свадьба' 'получение дополнительного образования' 'покупка своего жилья'
 'операции с недвижимостью' 'получение высшего образования'
 'свой автомобиль' 'сделка с автомобилем' 'профильное образование'
 'высшее образование' 'покупка жилья для сдачи' 'на покупку автомобиля'
 'ремонт жилью' 'заняться высшим образованием']


**purpose** содержит много семантически схожих целей, в дальнейшем необходимо категоризовать этот столбец

### Вывод

Датафрейм содержит пропущенные данные в столбцах **days_employed, total_income, dob_years, gender**. Необходимо разобраться в причинах их возникновения и понять, нужно ли их заполнять, и если нужно, то какими значениями. Также необходимо привести **education, family_status** к нижнему регистру и выполнить категоризацию столбца **purpose**.


## Предобработка данных

### Обработка дубликатов

In [14]:
### перевод столбца education в нижний регистр
data['education'] = data['education'].str.lower()
print(data['education'].unique())

['высшее' 'среднее' 'неоконченное высшее' 'начальное' 'ученая степень']


In [15]:
### перевод столбца family_status в нижний регистр
data['family_status'] = data['family_status'].str.lower()
print(data['family_status'].unique())

['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'не женат / не замужем']


In [16]:
### удаление полных дубликатов
print('полные дубликаты: {}'.format(data.duplicated().sum()))
data = data.drop_duplicates().reset_index(drop = True)
print(data.info())

полные дубликаты: 71
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21454 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21454 non-null  int64  
 3   education         21454 non-null  object 
 4   education_id      21454 non-null  int64  
 5   family_status     21454 non-null  object 
 6   family_status_id  21454 non-null  int64  
 7   gender            21454 non-null  object 
 8   income_type       21454 non-null  object 
 9   debt              21454 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21454 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None


### Вывод

Обработка дубликатов перед обработкой пропусков позволит избавиться от ряда проблем, которые могут возникнуть при дальнейшей обработке пропусков. 

Были приведены в нижний регистр значения стоблцов **education** и **family_status**, а затем были удалены полные дубликаты - 71 штука.
Возникновение  дубликатов различного регистра могло возникнуть при некорректном вводе данных, полные дубликаты могли стать следствием сбоя записи информации.

### Обработка пропусков

In [17]:
def avg_dob_years(row):
    income_type = row['income_type']
    mean_dob_years = row['dob_years']
    temp = row['dob_years']
    if temp == 0:
        mean_dob_years = data[(data['income_type'] == income_type)]['dob_years'].mean()
    return mean_dob_years

data['dob_years'] = data.apply(avg_dob_years, axis = 1)

Обработка 0-вых значений в стоблце **dob_years**. Для заполнения этого столбца используется функция avg_dob_years, которая заменяет 0 в возрасте клиента, на среднее значение возраста в зависимости от категории в **income_type**. 

In [18]:
def age_cat(years):
    AGE_YOUNG = 27
    AGE_MED = 45
    AGE_SENIOR = 60  
    if AGE_YOUNG > years:
        return 'молодежь'
    elif AGE_YOUNG <= years < AGE_MED:
        return 'средний возраст'
    elif AGE_MED <= years < AGE_SENIOR:
        return 'взрослый возраст'
    else:
        return 'пенсионер'


def avg_total_income(row):
    gender = row['gender']
    age_group = row['age_group']
    education_id = row['education_id']
    mean_income = row['total_income']
    temp = row['total_income']
    if math.isnan(temp):
        mean_income = data[(data['gender'] == gender)&
                                    (data['age_group'] == age_group)&
                                    (data['education_id'] == education_id)
            
        ]['total_income'].mean()
    return mean_income

In [19]:
data['age_group'] = data['dob_years'].apply(age_cat)
data['total_income'] = data.apply(avg_total_income, axis = 1)

**Обработка пропущенных значений в total_income**. Для заполнения этого столбца производится категоризация данных по возрасту клиента: так функция **age_cat** - добавляет новый столбец **age_group** в исходные данные, которые категоризует данные по возрасту клиента(В качестве порогов были выбраны 27, 45, 60 лет, которые разбивают исходные данные на 4 категории по возрасту:  молодежь, средний возраст, взрослый возраст, пенсионер.


Далее **avg_total_income** - заполняет пропущенные значения в total_income, средним по стобцам gender, age_group, education_id. На мой взгляд такая функция более точно заполнит значения total_income, чем среднее или медианное значение для всего набора данных без выбора категорий клиентов, т.к. учтет больше данные о каждом клиенте.

In [20]:
data['days_employed'] = abs(data['days_employed'])
print('mean_value:', data['days_employed'].mean())
print('median_value:', data['days_employed'].median())
print('min_value:', data['days_employed'].min())
print('max_value:', data['days_employed'].max())
print('amount_of_values > median_value:', data[(data['days_employed'] > data['days_employed'].median())]['days_employed'].count())
print('amount_of_values > mean_value:', data[(data['days_employed'] > data['days_employed'].mean())]['days_employed'].count())
data['days_employed'] = data['days_employed'].fillna(data['days_employed'].median())

mean_value: 66914.72890682236
median_value: 2194.220566878695
min_value: 24.14163324048118
max_value: 401755.40047533
amount_of_values > median_value: 9675
amount_of_values > mean_value: 3445


Обработка пропущенных значений в **days_employed**. Медиана точнее сможет описать пропущенные значения, из-за наличия выбросов в большую сторону в этом столбце у некоторых клиентов, но сперва необходимо изменить значения в этом столбце на положительные величины.

In [21]:
print(data[(data['children'] == -1)]['children'].count())
print(data[(data['children'] == 20)]['children'].count())
data['children'] = data['children'].replace(-1,0)
print(data[(data['children'] == -1)]['children'].count())

47
76
0


Обработка -1 в столбце **children**. Замена -1 на 0, 20 осталось и означает, что кол-во детей более 5.

In [22]:
data['gender'] = data['gender'].replace('XNA','F')
print(data['gender'].unique())

['F' 'M']


Замена одного значения в столбце **gender** с XNA на F. В данном случае это не сыграет роли на конечном результате, но если бы было больше неизвестных значений пришлось бы придумывать метод их заполнения.

### Вывод

Выполнены заполнения пропусков в столбцах датафрейма: **gender, children, total_income, dob_years, days_employed**. Добавлен столбец **age_group** с категориями клиентов по возрасту. 

### Замена типа данных

In [23]:
print(data.info())
data['dob_years'] = data['dob_years'].astype('int')
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')
print(data.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21454 non-null  int64  
 1   days_employed     21454 non-null  float64
 2   dob_years         21454 non-null  float64
 3   education         21454 non-null  object 
 4   education_id      21454 non-null  int64  
 5   family_status     21454 non-null  object 
 6   family_status_id  21454 non-null  int64  
 7   gender            21454 non-null  object 
 8   income_type       21454 non-null  object 
 9   debt              21454 non-null  int64  
 10  total_income      21454 non-null  float64
 11  purpose           21454 non-null  object 
 12  age_group         21454 non-null  object 
dtypes: float64(3), int64(4), object(6)
memory usage: 2.1+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 13 columns):
 #   Colum

### Вывод

 Замена типа данных столбцов **dob_years, days_employed, total_income** с вещественных float на int. Для более удобной работы. Использовался метод .astype(int)

### Лемматизация

In [24]:
from collections import Counter
from pymystem3 import Mystem
m = Mystem()
counter_lemmas = Counter(m.lemmatize(data.loc[0]['purpose']))
print(m.lemmatize(data.loc[0]['purpose']))
for k in range(1,len(data)):
    lemmas = m.lemmatize(data.loc[k]['purpose'])
    counter_lemmas_temp = Counter(lemmas)
    counter_lemmas = counter_lemmas + counter_lemmas_temp
print(counter_lemmas)
all_purposes = ['недвижимость','автомобиль','образование','свадьба','жилье']


['покупка', ' ', 'жилье', '\n']
Counter({' ': 33570, '\n': 21454, 'недвижимость': 6351, 'покупка': 5897, 'жилье': 4460, 'автомобиль': 4306, 'образование': 4013, 'с': 2918, 'операция': 2604, 'свадьба': 2324, 'свой': 2230, 'на': 2222, 'строительство': 1878, 'высокий': 1374, 'получение': 1314, 'коммерческий': 1311, 'для': 1289, 'жилой': 1230, 'сделка': 941, 'дополнительный': 906, 'заниматься': 904, 'проведение': 768, 'сыграть': 765, 'сдача': 651, 'семья': 638, 'собственный': 635, 'со': 627, 'ремонт': 607, 'подержанный': 486, 'подержать': 478, 'приобретение': 461, 'профильный': 436})


Были получены леммы для столбца **purpose**. Из результатов видно, что цели кредитов можно разбить на несколько основных категорий: *Недвижимость, автомобиль, образование, свадьба, жилье*

In [25]:
def lemmatizing_purpose(text):
    lem_text = m.lemmatize(text)
    purposes = (set(lem_text) & set(all_purposes))
    str_purposes = ''.join(str(s) for s in purposes)
    return str_purposes

In [26]:
data['lem_purposes'] = data['purpose'].apply(lemmatizing_purpose)
data['lem_purposes'] = data['lem_purposes'].replace('жилье','недвижимость')
data['lem_purposes'].value_counts()

lem_purposes
недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: count, dtype: int64

Получен новый столбец **lem_purposes** который содержит категорию цели кредита.

### Вывод

Произведена лемматизация целей кредита. Основные цели были определены по результам лемматизации столбца целей(Недвижимость, жилье, автомобиль, образование и свадьба), жилье заменено на недвижимость. Ремонт также был включен в категорию недвижимость. Далее был добавлен столбец **lem_purposes** в котором записаны основные цели: Недвижимость, автомобиль, образование и свадьба.

### Категоризация данных

#### Категоризация total_income

In [27]:
print('mean_value:', data['total_income'].mean())
print('median_value:', data['total_income'].median())
print('min_value:', data['total_income'].min())
print('max_value:', data['total_income'].max())
print('amount_of_values > mean_value:', data[data['total_income'] > data['total_income'].mean()]['total_income'].count())
print('amount_of_values > 100000:', data[data['total_income'] < 100000]['total_income'].count())
print('amount_of_values > 250000:', data[data['total_income'] > 250000]['total_income'].count())

mean_value: 167438.8279108791
median_value: 144904.0
min_value: 20667
max_value: 2265604
amount_of_values > mean_value: 8424
amount_of_values > 100000: 4463
amount_of_values > 250000: 2906


Идея заключается в том, чтобы разбить клиентов по категориям в зависимости от уровня дохода по крупным категориям.

In [28]:
def total_income_cat(income):
    LOW_INCOME = 80000
    AVG_INCOME = 145000
    HIGH_INCOME = 250000
    if LOW_INCOME > income:
        return '<80000'
    if LOW_INCOME <= income < AVG_INCOME:
        return '80000-145000'
    if AVG_INCOME <= income < HIGH_INCOME:
        return '145000-250000'
    else:
        return '>250000'

In [29]:
data['income_cat'] = data['total_income'].apply(total_income_cat)
print(data.groupby('income_cat')['income_cat'].count())

income_cat
145000-250000    7812
80000-145000     8460
<80000           2276
>250000          2906
Name: income_cat, dtype: int64


В качестве пороговых значений для **total_income** клиентов были выбраны 80000, 145000, 250000. Соответственно было получено 4 категории клиентов. Средний уровень доходаи уровень выше среднего имеют наибольшее кол-во клиентов - эти категории получились примерно равными по кол-ву клиентов, в то время как маленький и высокий доход имеют меньшее кол-во клиентов, но их доли от общего кол-ва примерно равны.

#### Категоризация children

In [30]:
def children_cat(children):
    many_children = 3
    if children == 0:
        return 'нет детей'
    if 0 < children < many_children:
        return 'есть менее трех детей'
    else:
        return 'многодетная семья'

In [31]:
data['children_cat'] = data['children'].apply(children_cat)
print(data.groupby('children_cat')['children_cat'].count())

children_cat
есть менее трех детей     6860
многодетная семья          456
нет детей                14138
Name: children_cat, dtype: int64


Категоризация **children** по кол-ву детей: менее 3 детей, нет детей и многодетная семья, если детей больше 3.

### Вывод

Для удобства были введены категории, разделяющие уровень дохода клиентов и кол-во детей у клиента. Очевидно, что средние группы по доходам должна быть наиболее распространненой в отличии от групп с малым и высоким доходом. По количеству детей группы были разбиты на тех, у кого нет детей вообще, тех у кого менее 3 детей и многодетные семьи с кол-вом детей более 3.
Ранее была использована категоризация данных по возрасту, чтобы заполнить значения total_income, возраст был разбит на несколько групп, которые могут быть использованы, как один из параметров для определения значений total_income.

## Ответить на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?
- Есть ли зависимость между семейным положением и возвратом кредита в срок?
- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
- Как разные цели кредита влияют на его возврат в срок?

### Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [32]:
data_children = data.pivot_table(index = 'children_cat', values = 'debt', aggfunc = ['count','sum'])
data_children.columns = ('total','with_debt')
#print('Процент задолженности по кредитам среди категорий клиентов, где:')
data_children['percent'] =  (data_children['with_debt'] / data_children['total'] * 100).round(2)
display(data_children.sort_values('percent'))




,total,with_debt,percent
children_cat,,,
нет детей,14138,1064,7.53
многодетная семья,456,39,8.55
есть менее трех детей,6860,638,9.30


----
По полученным результатам можно сделать вывод, что процент клиентов с долгами по кредитам больше, у тех клиентов, у которых есть дети. Это хорошо видно по категориям клиентов у которых нет детей - 7.53% и есть менее трех детей - 9.30%. Про категорию клиентов с многодетными семьями нельзя сделать четкого вывода, т.к. кол-во таких клиентов существенно меньше кол-ва клиентов в других категорий.

### Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [33]:
data_family = data.pivot_table(index = 'family_status', values = 'debt', aggfunc = ['count','sum'])
data_family.columns = ('total','with_debt')
print('Процент задолженности по кредитам среди категорий клиентов в зависимости от их семейного положения:')
data_family['percent'] = (data_family['with_debt'] / data_family['total'] * 100).round(2)
display(data_family.sort_values('percent'))


Процент задолженности по кредитам среди категорий клиентов в зависимости от их семейного положения:


,total,with_debt,percent
family_status,,,
вдовец / вдова,959,63,6.57
в разводе,1195,85,7.11
женат / замужем,12339,931,7.55
гражданский брак,4151,388,9.35
не женат / не замужем,2810,274,9.75


Лидерами по проценту должников это категории клиентов,в которых клиент не женат / не замужем или состоит в гражданском браке. Вдовцы и вдовы, как и те, кто в разводе замыкают этот рейтинг. Но стоит отметить, что категория женатых граждан слишком велика по сравнению с остальными категориями, поэтому представленный результат может быть не информативен и для ответа на вопрос о платежеспособности клиента и имеет смысл обратиться к другим характеристикам.

### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [34]:
data_income = data.pivot_table(index = 'income_cat', values = 'debt', aggfunc = ['count','sum'])
data_income.columns = ('total','with_debt')
print('Процент задолженности по кредитам среди категорий клиентов в зависимости от уровня их дохода:')
data_income['percent'] = (data_income['with_debt'] / data_income['total'] * 100).round(2)
display(data_income.sort_values('percent'))

Процент задолженности по кредитам среди категорий клиентов в зависимости от уровня их дохода:


,total,with_debt,percent
income_cat,,,
>250000,2906,198,6.81
<80000,2276,174,7.64
80000-145000,8460,711,8.40
145000-250000,7812,658,8.42


Видно, что при высоком доходе и наименьшем доходе процент клиентов с задолженностями меньше, по сравнению с клиентами из средних групп дохода. Для средних групп в свою очередь проценты почти равны.

### Как разные цели кредита влияют на его возврат в срок?

In [35]:
data_purpose = data.pivot_table(index = 'lem_purposes', values = 'debt', aggfunc = ['count','sum'])
data_purpose.columns = ('total','with_debt')
print('Процент задолженности по кредитам среди категорий клиентов в зависиомсти от целей кредита:')
data_purpose['percent'] = (data_purpose['with_debt'] / data_purpose['total'] * 100).round(2)
display(data_purpose.sort_values('percent'))

Процент задолженности по кредитам среди категорий клиентов в зависиомсти от целей кредита:


,total,with_debt,percent
lem_purposes,,,
недвижимость,10811,782,7.23
свадьба,2324,186,8.00
образование,4013,370,9.22
автомобиль,4306,403,9.36


Наиболее высокий процент по задолженностям среди клиентов, которые берут кредит с целью покупки автомобиля, наименьший у тех, кто берет кредит на покупку недвижимости, причем кол-во клиентов, которые обратились с целью покупки недвижимости больше, чем кол-во клиентов с другими целями, следовательно этому результату можно и доверять больше.

## Общий вывод

Были обработаны данные представленной таблицы. Из нее удалены дубликаты, заполненны  пропущенные/потерянные значения различными способами, выбор которых зависил от вопросов задачи и свойств данных таблицы. Данные категоризиваны по нескольким харакетиристикам для того, чтобы ответить на вопросы задачи. В конце выполнения представлены ответы на поставленные вопросы и представлены сводные таблицы результатов.